# Hospital Performance Analysis

## Introduction and Background

The goal of this analysis is to evaluate the performance of a fictional hospital across key metrics. These insights will then be leveraged to generate recommendations for hospital administration. 

This project uses synthetic data. The dataset used for analysis contains weekly aggregated service-level data for a fictional hospital over the course of one year. The hospital offers four services: emergency, general medicine, ICU, and surgery. Each row in the dataset represents a single service for a single week.

This analysis will focus on the following four key performance metrics:

1. Bed Utilization Rate
2. Patients Refused Services
3. Patient Satisfaction
4. Staff Morale

This analysis will explore how these metrics vary across hospital services throughout the year and the impact of events.

## Initial Data Exploration

In preparing the data, we add the following columns:

* percent_patients_refused: percent of patients who request services and are refused services
* utilization_rate: percent of beds occupied
* utilization_level: categorizes utilization_rate as low, ideal, high, or critical
    * low: utilization_rate less than 0.7
    * ideal: utilization_rate between 0.7 and 0.85
    * high: utilization_rate between 0.85 and 1.0
    * critical: utilization_rate of exactly 1.0

These utilization levels were chosen for multiple reasons. A utilization rate of 85% is commonly considered a critical threshold (see [Bagust et al., BMJ 1999](https://doi.org/10.1136/bmj.319.7203.155)). Utilization above 85% may cause a significant strain on hospital resources. A utilization rate of 100% indicates that the hospital is at maximum capacity. Conversely, a low utilization rate may indicate inefficient allocation of resources and result in lower profit.

In [ ]:
# SET UP ENVIRONMENT

library(tidyverse)
library(reshape2)

list.files(path = "../input")

# LOAD AND PREPARE DATA

services_weekly <- read_csv("/kaggle/input/hospital-beds-management/services_weekly.csv")

services_weekly <- services_weekly %>%
    mutate(percent_patients_refused = patients_refused / patients_request,
           utilization_rate = patients_admitted / available_beds,
           utilization_level = cut(utilization_rate,
                                  breaks = c(0, 0.7, 0.85, 1, Inf),
                                  labels = c("low", "ideal", "high", "critical"),
                                  right = FALSE))

head(services_weekly)

We begin the initial data exploration by visualizing trends in all four key metrics over the course of the year.

In [ ]:
# TIME SERIES OF KEY METRICS

services_weekly %>%
    select(service, utilization_rate, percent_patients_refused,
           patient_satisfaction, staff_morale, month) %>%
    group_by(service, month) %>%
    summarise("Utilization Rate (monthly avg)" = mean(utilization_rate),
              "Patients Refused (monthly avg percent)" = mean(percent_patients_refused),
              "Patient Satisfaction (monthly avg)" = mean(patient_satisfaction),
              "Staff Morale (monthly avg)" = mean(staff_morale)) %>%
    mutate(service = gsub("_", " ", service)) %>%
    melt(id.var = c("service", "month"))  %>%
    ggplot(aes(x = month, y = value, color = service)) +
    geom_line() +
    facet_wrap(~ variable, scale = "free") +
    theme_bw() +
    theme(legend.position = "top",
          legend.justification = "left",
          legend.title = element_blank()) +
    scale_fill_brewer(palette = "Set2") +
    scale_x_continuous(breaks = seq_along(month.abb), labels = month.abb) +
    labs(title = "Monthly Trends in Key Metrics", x = "", y = "")

While these charts demonstrate clear and significant variation in performance metrics over the course of the year, there are no clear trends in these variations at this stage in the analysis.

Next, we visualize the distribution of all four key metrics across hospital services.

In [ ]:
# BOXPLOTS OF KEY METRICS

services_weekly %>%
    select(service, utilization_rate, percent_patients_refused, 
           patient_satisfaction, staff_morale) %>%
    mutate(service = gsub("_", " ", service)) %>%
    rename("Utilization Rate" = utilization_rate,
           "Patients Refused Services (percent)" = percent_patients_refused,
           "Patient Satisfaction" = patient_satisfaction,
           "Staff Morale" = staff_morale) %>%
    melt(id.var = "service") %>%
    ggplot(aes(x = variable, y = value, fill = service)) +
    geom_boxplot(position = position_dodge(width = 1)) +
    facet_wrap(~ variable, scale = "free") +
    theme_bw() +
    theme(axis.text.x = element_blank(),
          legend.position = "top",
          legend.justification = "left",
          legend.title = element_blank()) +
    scale_fill_brewer(palette = "Set2") +
    labs(title = "Distribution of Key Metrics Across Hospital Services", 
         x = "", y = "")

These boxplots reveal the following insights.
* Utilization rate was consistently quite high. Emergency services had a utilization rate of 100% in all weeks. General medicine and surgery had a median utilization rate of 100%.
* Emergency services had the highest proportion of patients refused services, followed by general medicine. The proportion of patients refused services was relatively low in ICU and surgery services, though with some variation over the course of the year.
* Patient satisfaction and staff morale varied considerably over the course of the year, but were relatively consistent across hospital services.

To explore the impact of events across hospital services, we build a summary table using these four key metrics and other key variables. Using this table, we visualize how events impact these variables.

In [ ]:
# SUMMARY TABLE

summary_df <- services_weekly %>%
    select(service, event, utilization_rate, patients_request, patients_refused, 
           percent_patients_refused, patient_satisfaction, staff_morale) %>%
    bind_rows(mutate(., service = "all_services")) %>%
    mutate(event = factor(event, levels = c("none", "flu", "donation", "strike"))) %>%
    group_by(service, event) %>%
    summarise(num_weeks = n(),
              median_utilization_rate = median(utilization_rate),
              median_patients_request = median(patients_request),
              median_patients_refused = median(patients_refused),
              median_percent_refused = median(percent_patients_refused),
              median_patient_satisfaction = median(patient_satisfaction),
              median_staff_morale = median(staff_morale)) %>%
    arrange(fct_relevel(service, c("emergency", "general_medicine", 
                                   "ICU", "surgery", "all_services")))

# LINE PLOTS OF IMPACT OF EVENTS ON KEY METRICS

summary_df %>%
    select(service, event, median_utilization_rate, median_percent_refused,
           median_patient_satisfaction, median_staff_morale) %>%
    filter(service != "all_services") %>%
    mutate(service = gsub("_", " ", service)) %>%
    rename("Utilization Rate (median)" = median_utilization_rate,
           "Patients Refused Services (median percent)" = median_percent_refused,
           "Patient Satisfaction (median)" = median_patient_satisfaction,
           "Staff Morale (median)" = median_staff_morale) %>%
    mutate(event = recode(event, "none" = "no event")) %>%
    melt(id.var = c("service", "event")) %>%
    ggplot(aes(x = event, y = value, color = service, group = service)) +
    geom_line() +
    facet_wrap(~ variable, scale = "free") +
    theme_bw() +
    theme(legend.position = "top",
          legend.justification = "left",
          legend.title = element_blank()) +
    scale_color_brewer(palette = "Set2") +
    labs(title = "Impact of Events on Key Metrics", x = "", y = "")

summary_df

These charts reveal the following insights:

* Flu epidemics increased utilization rate and the proportion of patients refused services hospital-wide.
* Flu epidemics decreased patient satisfaction in ICU and surgery services.
* Donations increased patient satisfaction and staff morale hospital-wide.
* Strikes decreased utilization rate and the proportion of patients refused services hospital-wide.
* Strikes decreased staff morale hospital-wide.


## Initial Insights

From this initial exploration, we gain some key insights about our four performance metrics.

Metric 1: Bed Utilization Rate
* Consistently very high utilization rate across hospital services
* For emergency services, utilization rate was 100% for all weeks of the year
* General medicine and surgery had a median utilization rate of 100%, meaning they were at maximum capicity for at least 50% of the year
* ICU had a median utilization rate of 92.6%, which is above the critical threshold of 85%
* For ICU services, flu epidemics are associated with a spike in utilization rate
* For general medicine and surgery, strikes are associated with a dip in utilization rate

Metric 2: Patients Refused Services
* Emergency services consistently refuse services to a large percentage of patients (median of 80.2%)
* ICU and surgery services generally refuse a much smaller percentage or patients (median of 0% for ICU, median of 6.9% for surgery)
* The percentage of patients refused services spikes considerably across all hospital services during flu epidemics

Metric 3: Patient Satisfaction
* Patient satisfaction is relatively consistent across hospital services with generally high scores
* ICU services had the highest median patient satisfaction (83 out of 100)
* Emergency services had the lowest median patient satisfaction (75.5 out of 100)
* There is no clear or consistent impact of events on patient satisfaction across hospital services

Metric 4: Staff Morale
* Staff morale is relatively consistent across hospital services with generally moderately high scores
* Staff morale improves during weeks with donations and worsens during weeks with strikes hospital-wide

Overall Insights
* Bed utilization is consistently very high across all hospital services
* Emergency services consistently underperform in metrics 1, 2, and 3
* Flu epidemics have a distinct negative impact on metrics 1 and 2

## Deeper Analysis

Since bed utilization is consistently very high across all hospital services, we begin by analyzing this trend further.

We create a stacked bar chart to better visualize the distribution of bed utilization levels across hospital services.

In [ ]:
# STACKED BAR CHART OF UTILIZATION LEVELS BY SERVICE

services_weekly %>%
    mutate(service = gsub("_", " ", service)) %>%
    ggplot(aes(x = service, fill = utilization_level)) +
    geom_bar(position = position_fill(reverse = TRUE)) +
    guides(fill = guide_legend(reverse = TRUE)) +
    theme_bw() +
    scale_fill_brewer(palette = "RdYlBu", direction = -1) +
    labs(title = "Bed Utilization Level by Hospital Service",
         x = "",
         y = "percent of weeks",
         fill = "utilization level")

It is immediately clear from this visualization that all services were at a critical utilization level (100% utilization rate) for a large portion of the year. Of particular concern, emergency services were at a critical utilization level every week of the year.

To further investigate factors that may lead to a high or critical utilization level in hospital services, we examine the relationship between utilization rate and the volume of patient requests.

In [ ]:
# SCATTERPLOT OF UTILIZATION RATE VS PATIENT REQUESTS

services_weekly %>%
    mutate(service = gsub("_", " ", service)) %>%
    ggplot(aes(x = patients_request, y = utilization_rate)) +
    geom_point() +
    facet_wrap(~ service, scale = "free_x") +
    theme_bw() +
    labs(title = "Utilization Rate vs. Patient Requests",
         x = "number of patient requests",
         y = "utilization rate")

# CORRELATION COEFFICIENTS

services_weekly %>%
    bind_rows(mutate(., service = "all_services")) %>%
    group_by(service) %>%
    summarise(cor_requests_utilization = cor(patients_request, utilization_rate)) %>%
    arrange(fct_relevel(service, c("emergency", "general_medicine", 
                                   "ICU", "surgery", "all_services")))

Patient request volume appears to be a driver of high and critical utilization rates across hospital services. For general medicine, ICU, and surgery services, there is a clear positive correlation between patient requests and utilization rate. Moreover, for all four hospital services, there is a threshold for patient requests beyond which utilization rate is consistently at maximum capacity.

Another relevant variable to consider is bed availability, since this is not a constant and varies from week to week across hospital services.

In [ ]:
# SCATTERPLOT OF UTILIZATION RATE VS AVAILABLE BEDS

services_weekly %>%
    mutate(service = gsub("_", " ", service)) %>%
    ggplot(aes(x = available_beds, y = utilization_rate)) +
    geom_point() +
    facet_wrap(~ service, scale = "free_x") +
    theme_bw() +
    labs(title = "Utilization Rate vs. Bed Availability",
         x = "number of available beds",
         y = "utilization rate")

# CORRELATION COEFFICIENTS

services_weekly %>%
    bind_rows(mutate(., service = "all_services")) %>%
    group_by(service) %>%
    summarise(cor_requests_utilization = cor(available_beds, utilization_rate)) %>%
    arrange(fct_relevel(service, c("emergency", "general_medicine", 
                                   "ICU", "surgery", "all_services")))

Bed availability does not appear to be a relevant driver of high and critical utilization rates, as there is no clear association between the two variables. This is confirmed by weak correlation coefficients and scatterplots showing no clear trend.

Finally, we noted in our initial exploration that flu epidemics appear to cause a spike in utilization rate and strikes appear to cause a dip in utilization rate. We further our analysis now by exploring the relationship between events and utilization rate further.

In [ ]:
# STACKED BAR CHART OF UTILIZATION LEVELS BY EVENT

services_weekly %>%
    mutate(event = factor(event, levels = c("none", "flu", "donation", 
                                            "strike", "all_events"))) %>%
    mutate(event = recode(event, "none" = "no event")) %>%
    ggplot(aes(x = event, fill = utilization_level)) +
    geom_bar(position = position_fill(reverse = TRUE)) +
    guides(fill = guide_legend(reverse = TRUE)) +
    theme_bw() +
    scale_fill_brewer(palette = "RdYlBu", direction = -1) +
    labs(title = "Bed Utilization Level by Event",
       x = "",
       y = "percent of weeks (all services)",
       fill = "utilization level")

# BAR CHART OF PATIENT REQUESTS BY EVENT

services_weekly %>%
    mutate(service = gsub("_", " ", service)) %>%
    group_by(service, event) %>%
    mutate(event = factor(event, levels = c("none", "flu", "donation",
                                            "strike", "all_events"))) %>%
    mutate(event = recode(event, "none" = "no event")) %>%
    summarise(median_requests = median(patients_request)) %>%
    ggplot(aes(x = event, y = median_requests,
               fill = reorder(service, median_requests, sum, decreasing = TRUE))) +
    geom_bar(stat = "identity", position = "dodge") +
    theme_bw() +
    scale_fill_brewer(palette = "Set2") +
    labs(title = "Impact of Events on Patient Requests",
         x = "",
         y = "number of patient requests (median)",
         fill = "hospital service")

Across all hospital services, patient requests increase during flu epidemics and decrease during strikes. Accordingly, bed utilization is consistently higher during flu epidemics and lower during strikes. Of particular note, during all weeks in which there was a flu epidemic, utilization rate was 100% (critical).

Flu epidemics are typically a seasonal occurrence. To examine this pattern, we create a timeseries plot to visualize the monthly prevalence of flu epidemics in the hospital.

In [ ]:
# TIME SERIES GRAPH OF FLU EPIDEMICS

services_weekly %>%
    group_by(month) %>%
    summarise(num_entries = n(), count_flu = sum(event == "flu")) %>%
    mutate(pct_flu_events = count_flu / num_entries) %>%
    ggplot(aes(x = month, y = pct_flu_events)) +
    geom_line() +
    scale_x_continuous(breaks = seq_along(month.abb), labels = month.abb) +
    theme_bw() +
    labs(title = "Seasonal Prevalence of Flu Epidemics",
         x = "", y = "prevalence of flu events (percent)")

This visualization clearly shows that the prevalence of flu epidemics varied over the course of the year, with no epidemics between the months of May and November and highest prevalence in January.

Having identified patient request volume as a potential driver of poor performance, we examine its relationship to a second key metric: the percent of patients refused service.

In [ ]:
# SCATTERPLOT OF NUM PATIENTS REFUSED VS AVAILABLE BEDS

services_weekly %>%
    mutate(service = gsub("_", " ", service)) %>%
    ggplot(aes(x = patients_request, y = patients_refused)) +
    geom_point() +
    facet_wrap(~ service, scale = "free") +
    theme_bw() +
    labs(title = "Patients Refused vs Patient Requests",
         x = "number of patient requests",
         y = "number of patients refused services")

# CORRELATION COEFFICIENTS

services_weekly %>%
    bind_rows(mutate(., service = "all_services")) %>%
    group_by(service) %>%
    summarise(cor_requests_refused = cor(patients_request, patients_refused)) %>%
    arrange(fct_relevel(service, c("emergency", "general_medicine", 
                                   "ICU", "surgery", "all_services")))

This analysis reveals that patient request volume is a very strong predictor of the number of patients who are refused hospital services. The scatterplots show a clear positive trend. This is confirmed by the correlation coefficients, which indicate a strong linear correlation between patient requests and patients refused services across all hospital services.

Overall, patient request volume is associated with performance across hospital services in two key metrics: bed utilization rate and patients refused services. Moreover, there is a clear association between flu epidemics and high patient request volume.

## Conclusions and Recommendations

The aim of this analysis was to evaluate the performance of the hospital and its four services (emergency, general medicine, ICU, and surgery) across the following four key metrics:

1. Bed Utilization Rate
2. Patients Refused Services
3. Patient Satisfaction
4. Staff Morale

The hospital generally performed well in metrics 3 and 4 (patient satisfaction and staff morale). However, metric 1 (bed utilization rate) emerged as an area of critical concern. Metric 2 (patients refused service) is another potential area for improvement. Emergency services showed especially poor performance in these metrics, and performance worsened across all services during flu epidemics.

Accordingly, we make the following recommendations to hospital administration:
1. Take steps to better prepare for flu season (typically October through May)
    * Anticipate a higher volumne of patient requests during flu epidemic events
    * Allocate resources to increase staffing and bed availability during flu epidemic events
2. Take steps to lower bed utilization year-round
    * Aim for a utilization rate under 85%
    * Research strategies for improving resource management and allocation
3. Take steps to improve outcomes in emergency services
    * Allocate more resources to emergency services to lower critically high bed utilization rates
    * Optimize patient flow in emergency services
    * Expedite care for lower-acuity patients to decrease the number of patients refused service